In [1]:
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
from sklearn.utils import resample
import tensorflow as tf

In [2]:
df = pd.read_csv("/content/Fake_Real.csv")
df.head()

,Unnamed: 0,text,subject,target
0,0,new york reuters us environmental group sierra...,politicsNews,True
1,1,washington reuters us air force asked industry...,politicsNews,True
2,2,saturday paul ryan posted photo instagram phot...,News,Fake
3,3,america keeps waiting word hillary indicted ob...,politics,Fake
4,4,religion peace ht weasel zippers,left-news,Fake


In [3]:
df = df.drop('Unnamed: 0', axis=1)# 1 for drop column
df = df.drop('subject', axis=1)# 1 for drop column
df.head()

,text,target
0,new york reuters us environmental group sierra...,True
1,washington reuters us air force asked industry...,True
2,saturday paul ryan posted photo instagram phot...,Fake
3,america keeps waiting word hillary indicted ob...,Fake
4,religion peace ht weasel zippers,Fake


In [4]:
df.isnull().sum()

,0
text,632
target,0


In [5]:
df = df.dropna()
df.isnull().sum()

,0
text,0
target,0


In [6]:
df['target'] = df['target'].map({'True': 0, 'Fake': 1})
df.head()

,text,target
0,new york reuters us environmental group sierra...,0
1,washington reuters us air force asked industry...,0
2,saturday paul ryan posted photo instagram phot...,1
3,america keeps waiting word hillary indicted ob...,1
4,religion peace ht weasel zippers,1


In [7]:
df['target'].value_counts()

,count
target,
1,22850
0,21416


In [ ]:
!python -m spacy download en_core_web_lg


In [9]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [10]:
doc = nlp("Natural Language Processing is a very popular topic in Machine Learning. It is very interesting")

doc[0].vector.shape

(300,)

In [11]:
df['vector'] = df['text'].apply(lambda text: nlp(text).vector)

In [12]:
df.head()

,text,target,vector
0,new york reuters us environmental group sierra...,0,"[-0.1670631, 0.15209822, -0.010812569, 0.05380..."
1,washington reuters us air force asked industry...,0,"[-0.07177868, 0.035780445, 0.14794463, 0.00469..."
2,saturday paul ryan posted photo instagram phot...,1,"[-0.10085299, 0.050738174, -0.017068217, 0.017..."
3,america keeps waiting word hillary indicted ob...,1,"[-0.104578726, 0.05843705, 0.02255843, 0.03185..."
4,religion peace ht weasel zippers,1,"[-0.07285362, 0.01179217, -0.072706595, 0.2278..."


In [22]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(df['vector'],df['target'],test_size=0.2,random_state=42)

# Convert the Series of arrays into a 2D NumPy array
X_train_2d = np.stack(X_train)
X_test_2d = np.stack(X_test)

In [23]:
X_train_2d[:10]

array([[-0.08772203,  0.12098263,  0.04515941, ..., -0.06294117,
        -0.00470237,  0.14209418],
       [-0.17184152,  0.13339184,  0.00060377, ...,  0.06341763,
        -0.01301223,  0.08579111],
       [-0.0800301 , -0.00972594,  0.06749333, ..., -0.01684754,
        -0.11055733,  0.10323656],
       ...,
       [-0.07973416,  0.13010554,  0.05239066, ...,  0.00695289,
        -0.08303253,  0.05349796],
       [-0.14295754,  0.11575934,  0.08717911, ...,  0.02814568,
        -0.06122342,  0.07885591],
       [-0.12335915,  0.10886062,  0.03155953, ..., -0.0494085 ,
         0.01918995, -0.00546381]], dtype=float32)

In [25]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=50,criterion='entropy')

model.fit(X_train_2d, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [26]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test_2d)

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      4273
           1       0.97      0.97      0.97      4581

    accuracy                           0.97      8854
   macro avg       0.97      0.97      0.97      8854
weighted avg       0.97      0.97      0.97      8854



In [29]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=5,metric='euclidean')

model.fit(X_train_2d, y_train)

KNeighborsClassifier(metric='euclidean')

In [30]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test_2d)

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.97      0.94      4273
           1       0.97      0.92      0.95      4581

    accuracy                           0.94      8854
   macro avg       0.95      0.95      0.94      8854
weighted avg       0.95      0.94      0.94      8854



In [31]:
df = pd.read_json('/content/News_Category_Dataset_v3.json')

df.head()

ValueError: Trailing data